In [40]:
from sklearn.neural_network import MLPClassifier
import numpy as np
import scipy
X1 = np.random.multivariate_normal([0, 0], [[1, 0], [0,1]], 500)
X2 = np.random.multivariate_normal([1, 1], [[1, 0], [0, 1]], 500)
X = np.vstack((X1, X2))
y1 = np.zeros(500)
y2 = np.ones(500)
y = np.vstack((y1,y2))

global W1
W1 = np.random.random((2,3))
global W2
W2 = np.random.random((3,5))
global W3
W3 = np.random.random((5,3))
global W4
W4 = np.random.random((3,1))

In [44]:
def MSE(y,t):
    loss = np.square(y-t)
    return np.sum(loss) / loss.shape[1]




def forward(x):
    global W1
    global W2
    global W3
    global W4
    H1 = np.matmul(x,W1)
    H1_out = scipy.special.expit(H1)
    H2 = np.matmul(H1_out,W2)
    H2_out = scipy.special.expit(H2)
    H3 = np.matmul(H2_out,W3)
    H3_out = scipy.special.expit(H3)
    y = np.matmul(H3_out,W4)
    y_out = scipy.special.expit(y)
    return y_out

In [45]:
def MSE_back(x,lr=0.001):
    global W1
    global W2
    global W3
    global W4
    
    H1 = np.matmul(x,W1)
    H1_out = scipy.special.expit(H1)
    H2 = np.matmul(H1_out,W2)
    H2_out = scipy.special.expit(H2)
    H3 = np.matmul(H2_out,W3)
    H3_out = scipy.special.expit(H3)
    y = np.matmul(H3_out,W4)
    y_out = scipy.special.expit(y)
    
    dE_W4 = np.zeros((3, ))
    for i in range(1000):
        delta3i = y_out[:, i][0] - y[i]
        part2 = scipy.special.expit(y_out[:, i][0])
        part2 = part2 * (1 - part2)
        part3 = H3_out[:, i]
        dE_W4 += delta3i * part2 * part3
    dE_W4 = np.expand_dims(dE_W4, axis=1)
    
    dE_W3 = np.zeros((5, 3))
    
    for i in range(1000):
        part1 = y_out[:, i][0] - y[i]
        part2 = scipy.special.expit(y_out[:, i][0])
        part2 = part2 * (1 - part2)
        
        delta3i = w4 * part1 * part2
        delta3i = np.squeeze(delta3i)
        part3 = scipy.special.expit(H3[:, i]) * (1 - scipy.special.expit(H3[:, i]))
        mid = np.multiply(delta3i, part3)
        mid = np.expand_dims(mid, axis=1)
        val = np.dot(mid, np.expand_dims(H2_out[:, i], axis=1).T)
        dE_W3 += val.T

    dE_W2 = np.zeros((3, 5))
    for i in range(1000):
        part1 = y_out[:, i][0] - y[i]
        part2 = scipy.special.expit(y_out[:, i][0])
        part2 = part2 * (1 - part2)
        
        delta3i = part1 * part2
        c = scipy.special.expit(H3[:, i]) * (1 - scipy.special.expit(H3[:, i]))
        c = np.expand_dims(c, axis=1)
        delta2i = np.multiply(w4 * delta3i, c)
        
        part3 = np.dot(w3, delta2i)
        d = scipy.special.expit(H2[:, i]) * (1 - scipy.special.expit(H2[:, i]))
        d = np.expand_dims(d, axis=1)
        e = np.multiply(part3, d)
        
        val = np.dot(e, np.expand_dims(H1_out[:, i], axis=1).T)
        dE_W2 += val.T
        
    dE_W1 = np.zeros((2, 3))
    for i in range(1000):
        part1 = y_out[:, i][0] - y[i]
        part2 = scipy.special.expit(y_out[:, i][0])
        part2 = part2 * (1 - part2)
        
        delta3i = part1 * part2
        c = scipy.special.expit(H3[:, i]) * (1 - scipy.special.expit(H3[:, i]))
        c = np.expand_dims(c, axis=1)
        delta2i = np.multiply(w4 * delta3i, c)
        
        p1 = np.dot(w3, delta2i)
        d = scipy.special.expit(H2[:, i]) * (1 - scipy.special.expit(H2[:, i]))
        d = np.expand_dims(d, axis=1)
        delta1i = np.multiply(p1, d)
        
        p2 = np.dot(w2, delta1i)
        e = scipy.special.expit(H1[:, i]) * (1 - scipy.special.expit(H1[:, i]))
        e = np.expand_dims(e, axis=1)
        
        delta = np.multiply(p2, e)
        val = np.dot(delta, np.expand_dims(X[i], axis=1).T)
        dE_W1 += val.T
        
        W4 = W4 - lr * dE_W4        
        W3 = W3 - lr * dE_W3        
        W2 = W2 - lr * dE_W2
        W1 = W1 - lr * dE_W1
        

In [46]:
loss = []
for i in range(500):
    MSE_back(X)
    y_pred = forward(X)
    loss.append(MSE(y, y_pred))
plt.plot(loss)

ValueError: operands could not be broadcast together with shapes (3,) (1000,) (3,) 